<img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/>

# Getting Started with the Arize Platform - Predicting Claims in Insurance

**In this walkthrough, we are going to investigate various performance related aspects of an insurance use-case. More specificlly we will be using the Arize platform to monitor claim prediction performance.**

You manage the Machine Learning models for an insurance company. You have spent a great deal of your time collecting data and traning models for best performance. With your models now in production you need tools to detect drift, identify any issues or get insights into how to improve your models. In this walkthrough we will look at a few scenarios common to an insurance use-case and more specifically looking at claims probability predictions versus actuals for a block of general liability insurance covering small businesses.

You will learn to:

1. Get training and production data into the Arize platform
2. Setup performance dashboards
3. Setup threshold alerts
4. Understand where the model is underperforming
5. Discover the root cause of issues

# Step 1. Getting the Data
First step is to load a preexisting dataset that represents a insurance use-case including training and production data. Using a preexisting dataset illustrates how simple it is to plug into the Arize platform.

In [1]:
!pip install -q arize

import pandas as pd
from arize.pandas.logger import Client, Schema
from arize.utils.types import Environments, ModelTypes

url = "https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance"
df = {env: pd.read_csv(f"{url}/{env}.zip") for env in ["feat", "train", "prod"]}
feature_column_names = df["feat"]["Feature"].tolist()

# Update prediction timestamps
from datetime import datetime

ts_shift = datetime.now().timestamp() - df["prod"]["prediction_ts"].max()
df["prod"].eval("prediction_ts = prediction_ts + @ts_shift", inplace=True)

print("✅ Data successfully loaded!")

✅ Data successfully loaded!


## Inspect the Data 

Take a quick look at the dataset. The data represents a model designed and trained to evaluate the probability that a user will file an insurance claim. The dataset contains one month of data and the performance will be evaluated by comparing:

*   **PREDICTION**: The probability of a fraud transaction predicted by the model 
*   **ACTUAL**: Fraud or not fraud based on ground truth collected by credit card users


In [2]:
df["train"][:10]

,Unnamed: 0,MSADSCR,NAICSDSCR,state,firm_size_bracket,established_business,experienced_business,safe_business,owner_gender,prediction_score,actual_score,actual_label,prediction_label,payroll_bracket,avg_payroll_bracket,risk_bracket,prediction_id
0,0,"Charlotte-Concord-Gastonia, NC-SC",Beverage and Tobacco Product Manufacturing,NC,10-19,N,N,Y,Male,0.141024,0,No Claims,Filed a Claim,"(0K, 100K]","(9K, 40K]",2,a66f36a7-f233-41ed-8e1f-128612e259e9
1,1,"Los Angeles-Long Beach-Anaheim, CA",Truck Transportation,CA,5-9,Y,N,Y,Male,0.063550,0,No Claims,Filed a Claim,"(0K, 100K]","(9K, 40K]",1,1c6bb26b-41c4-45c5-9dd3-15d5059819c7
2,2,"Chicago-Naperville-Elgin, IL-IN-WI",Couriers and Messengers,IL,<5,Y,N,N,Male,0.035389,0,No Claims,No Claims,"(0K, 100K]","(40K, 140K]",1,6ddb74ee-8851-4bab-a7fc-1b6b89a3b835
3,3,"Chicago-Naperville-Elgin, IL-IN-WI",Couriers and Messengers,IL,<5,N,Y,Y,Male,0.051116,0,No Claims,Filed a Claim,"(0K, 100K]","(40K, 140K]",1,caca9454-ff78-43a0-be96-e7e25bd4981b
4,4,"Atlanta-Sandy Springs-Roswell, GA",Heavy and Civil Engineering Construction,GA,5-9,Y,Y,Y,Male,0.067952,0,No Claims,No Claims,"(100K, 600K]","(40K, 140K]",1,9ae7f989-0ddb-4f7d-87ff-48ee36c8fe5b
5,5,"Boston-Cambridge-Newton, MA-NH",Transit and Ground Passenger Transportation,MA,<5,N,Y,N,Female,0.012482,0,No Claims,No Claims,"(0K, 100K]","(9K, 40K]",0,9902c4b9-7495-45c8-b2fa-a0a52b442d53
6,6,"Charlotte-Concord-Gastonia, NC-SC",Heavy and Civil Engineering Construction,NC,10-19,Y,N,N,Female,0.143597,0,No Claims,Filed a Claim,"(100K, 600K]","(9K, 40K]",2,aec98497-7e5f-4fd0-be4b-ef0e0fb71cf3
7,7,"Dallas-Fort Worth-Arlington, TX",Truck Transportation,TX,20-49,Y,Y,Y,Male,0.278802,0,No Claims,Filed a Claim,"(600K, 6300K]","(9K, 40K]",3,7e7fa997-1fed-4025-85db-a98a7f80c984
8,8,"New York-Newark-Jersey City, NY-NJ-PA",Waste Management and Remediation Services,NY,<5,Y,Y,Y,Male,0.012342,0,No Claims,Filed a Claim,"(0K, 100K]","(9K, 40K]",0,751669f3-e94a-4fe6-8735-c89c44420b18
9,9,"New York-Newark-Jersey City, NY-NJ-PA",Waste Management and Remediation Services,NY,<5,Y,N,Y,Female,0.037841,0,No Claims,No Claims,"(0K, 100K]","(9K, 40K]",1,da5d6b4c-fe89-4afd-abd6-e56c32088a95


# Step 2. Sending Data into Arize 💫

Now that we have our data configured, we are ready to integrate into Arize. We do this by logging (sending) important data we want to analyze to the platform. There, the data will be easily visualized and investigated to source our problem.

For our model, we are going to log:

*   feature data
*   predictions
*   actuals

## Import and Setup Arize Client

The first step is to setup our Arize client. After that we will log the data.

First, copy the Arize `API_KEY` and `SPACE_KEY` from your admin page linked below! Copy those over to the set-up section. We will also be setting up some metadata to use across all logging.



In [48]:
SPACE_KEY = "SPACE_KEY"
API_KEY = "API_KEY"

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

# model_id the model name that will show up in Arize
model_id = "insurance-use-case"
model_version = "v1.0"  # Version of model - can be any string

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Arize setup complete!")

✅ Arize setup complete!


## Log Training & Production Data
Now that our Arize client is setup, let's go ahead and log all of our data to the platform. 

Arize lets you log predictions first and actuals later, and will match them up by the same `prediction_id` when both are ready. But for the purpose of this demo, we'll log predictions and actual together.

For details on how **`arize.pandas.logger`** can log predictions and actuals separately, visit out documentations page below.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://docs.arize.com/arize/sdks-and-integrations/python-sdk/arize.pandas)

Key parameters:

| Schema Argument Name | Description | 
| --- | --- |
| prediction_id_column_name| list of unique ids you can use to use to match each record 
| timestamp_column_name| timestamps for when predictions were made 
| prediction_label_column_name| tells Arize which column contains the predictions ("Filed a Claim"/"No Claims")
| prediction_score_column_name| tells Arize which column contains the predicted probability that a user "Filed a Claim" (probability 0 to 1 where 1 = "Filed a Claim")
| actual_label_column_name| tells Arize which column contains the actual results from field data ("Filed a Claim"/"No Claims")
| actual_score_column_name | same as the actual label but expresed as 0 or 1
| feature_column_names| names of the columns representing features 


We will use [ModelTypes.SCORE_CATEGORICAL](https://docs.arize.com/arize/product-guides-1/models/model-types) to log our model into Arize. 

In [49]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    prediction_id_column_name="prediction_id",
    prediction_label_column_name="prediction_label",
    prediction_score_column_name="prediction_score",
    actual_label_column_name="actual_label",
    actual_score_column_name="actual_score",
    feature_column_names=feature_column_names,
)

# Log Pandas DataFrame
response = arize_client.log(
    dataframe=df["train"],
    schema=schema,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.TRAINING,
)

# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(f"logging failed with response code {response.status_code}, {response.text}")
else:
    print(
        f"✅ You have successfully logged {len(df['train']):,} training data points to Arize"
    )

# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="prediction_label",
    prediction_score_column_name="prediction_score",
    actual_label_column_name="actual_label",
    actual_score_column_name="actual_score",
    feature_column_names=feature_column_names,
)

# Log Pandas DataFrame
response = arize_client.log(
    dataframe=df["prod"],
    schema=schema,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.PRODUCTION,
)

# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(f"logging failed with response code {response.status_code}, {response.text}")
else:
    print(
        f"✅ You have successfully logged {len(df['prod']):,} production data points to Arize"
    )

✅ You have successfully logged 47,838 training data points to Arize
✅ You have successfully logged 201,389 production data points to Arize


# Coffee Time! ☕️
Note that the Arize performs takes about 10 minutes to index the data. While the model should appear immediately, the data will not show up untill the indexing is done. Feel free to go grab a cup of coffee as Arize works its magic! 🔮

**⚠️ DON'T SKIP:**
In order to move on to the next step, make sure your actuals and training/validation sets are loaded into the platform. To check:
1. Naviate to models from the left bar, locate and click on model **insurance-use-case**
2. On the **Overview Tab** and make sure you see the actuals as shown below.
3. Actual data will show up under **Model Health**. Once the number changes from **0 Actuals** to **Actuals** (with summary statistics such as cardinality listed in the tables), your production actuals will have been fully recorded on Arize!
4. Verify the list of features under **Model Health**. 

<img src="https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/initial_overview.png" width="800"/>

Arize can automatically configure monitors that are best suited to your data. From the banner at the top of the screen, simply click **Set up Model** then select **Training Version v1.0** and click **NEXT**. Select **Filed a Claim** as **Positive Class**. Click **NEXT** three more times with Arize's default recommended settings.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/use_cases/img/initial_setup_banner.png)

You will now see that the baseline has been set and **Drift**, **Data Quality**, and **Performance** monitors have been created!!!

<img src="https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/initial_setup_monitors.png" width="300"/>

# Step 3. Analyzing feature drift
During intial model setup, Arize automatically calculates drift (and sets up drift monitors) for each feature available in your dataset. Drift is defaulted to a statistical measure of Population Stability Index (PSI) over a given period. We will use these graphs to monitor overall trends. From the model overview page, select the **MSADSCR** feature and you should notice the change in MSA distribution. Use the PSI graph to select a period of interest. We see that Los Angeles and Dallas have been gaining more businesses than they had in the training dataset.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/MSA_Drift.png)

# Step 4. Analyzing and detecting missing data
In the production data, you might have already noticed some drift on the **experienced_business** feature. To further investigate, from the **experienced_business** feature page, navigate to the **Data Quality** tab. Here we notice missing values are being introduced in the **experienced_business** feature using the Percent Empty graph. Make sure to view the last 30 days by selecting the correct range in the top right corner of the screen. Arize keeps tracks of feature cardinality and empty/missing values over time to help pin-point when the issue started.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/Percent_Empty.png)

To automatically be notified of missing values getting introduced in production, create a custom monitor for feature **experienced_business**. From the model **Monitors** tab click **New Monitor** and chose **Data Quality Monitor**. Enter these parameters:

* Dimension -> Feature --> experienced_business
* Aggregation -> Percent Empty
* Set a low threshold

![image.png](https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/Percent_Empty_Monitor.png)


**Congratulations!** 🎉   You have created your first monitor that will keep you alerted of any changes in your production environment.

# Step 6. Analyzing model performance
The screen capture below shows how to set up a **Model Performance Dashboard** to monitor the expected vs actual probability of filing a claim.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/perfdash.gif)

The screen capture below shows how to set up a **Feature Performance Dashboard** to drill down into root causes of model underperformance.

![image.png](https://storage.googleapis.com/arize-assets/fixtures/use_cases/insurance/img/heatdash.gif)

---
# Conclusion

In this walkthrough we've shown how Arize can be used to log prediction data for a model, pinpoint model performance degredation, and set up monitors to catch future issues. We have been able to identify 3 areas of concern:

1. Production distribution has drifted from training data.
2. Missing values are appearing in production. Further investigation is needed.
3. The model is underperforming in certain segments. Retraining may be needed.

Summary of the analysis and tools:

Concern                   | Detection                       | Root Cause Analysis Tools
--------------------------|---------------------------------|----
Data Drift  | Use a **Drift Monitor**             | - Built-in distribution graphs on each feature<br>- Review distribution at different time intervals
Bad  data            | Use a **Data Quality Monitor**      | - Built-in cardinality graphs<br>- Built-in Empty over Time graph<br>- Create Data Quality Monitors
Inaccuracy                | Use a **Model Performance Monitor**<br>Use a **Feature Performance Heatmap** | - Actual vs Expected timeseries widget<br>- Filter on features and values to narrow down issues

<br>

Though we covered a lot of ground, this is just scratching the surface of what the Arize platform can do. We urge you to explore more of Arize, either on your own or through one of our many other tutorials.

### About Arize
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
